In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import json
import numpy as np
import pandas as pd
from tensorflow.keras import layers
from tqdm import tqdm
import matplotlib.pyplot as plt
from tqdm import tqdm
import foolbox as fb


tf.compat.v1.enable_eager_execution()
tf.keras.backend.clear_session()  # For easy reset of notebook state.

# Prune, Train Attack Pipeline

In [7]:
pgd_success_rates = []
cw_success_rates = []
all_accuracies = []
for j in tqdm(range(1)):
    accuracies = []
    pgd_success_rate = []
    cw_success_rate = []
    compression_rates = [1, 2, 4, 8, 16, 32]
    pruning_ratios = [1-1/x for x in compression_rates]
    for index, pruning_ratio in tqdm(enumerate(pruning_ratios)):
        model = tf.keras.models.load_model('../saved-models/mini-pipeline-baseline-model')
        for i in range(index + 1):
            if i != index:
                pruned_weights = prune_weights(model, pruning_ratios[i])
                model.set_weights(pruned_weights)
                model = train_model(model, to_convergence=False)
            if i == index:
                print('final pruning and eval')
                pruned_weights = prune_weights(model, pruning_ratios[i])
                model.set_weights(pruned_weights)
                model = train_model(model, to_convergence=True)
                accuracies.append(model.evaluate(x_test, y_test, verbose=0))
                pgd_success_rate.append(pgd_attack(model))
                cw_success_rate.append(cw2_attack(model))
                
    all_accuracies.append(accuracies)
    pgd_success_rates.append(pgd_success_rate)
    cw_success_rates.append(cw_success_rate)
#write to csv and json
pd.DataFrame(all_accuracies).to_csv('accuracies.csv',index=False)
with open('accuracies.json', 'w') as f:
    json.dump(all_accuracies, f)
    
pd.DataFrame(pgd_success_rates).to_csv('pgd_success.csv',index=False)
with open('pgd_success.json', 'w') as f:
    json.dump(pgd_success_rates, f)
    
pd.DataFrame(cw_success_rates).to_csv('cw2_success.csv',index=False)
with open('cw2_success.json', 'w') as f:
    json.dump(cw_success_rates, f)

  0%|          | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]

final pruning and eval
Epoch 1/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4982 - accuracy: 0.9647 - val_loss: 1.4951 - val_accuracy: 0.9669
Epoch 2/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4905 - accuracy: 0.9721 - val_loss: 1.4918 - val_accuracy: 0.9707
Epoch 3/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4866 - accuracy: 0.9751 - val_loss: 1.4872 - val_accuracy: 0.9749
Epoch 4/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4832 - accuracy: 0.9786 - val_loss: 1.4912 - val_accuracy: 0.9701
Epoch 5/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4813 - accuracy: 0.9805 - val_loss: 1.4903 - val_accuracy: 0.9711
Epoch 6/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4804 - accuracy: 0.9812 - val_loss: 1.4884 - val_accuracy: 0.9730
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.



1it [13:58, 838.68s/it]

self.p 2
flatten(p) TensorFlowTensor(<tf.Tensor: shape=(500, 784), dtype=float32, numpy=
                 array([[0.0000000e+00, 9.7081065e-04, 5.3644180e-07, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        ...,
                        [0.0000000e+00, 0.0000000e+00, 1.3113022e-05, ..., 2.0861626e-07,
                         2.8789043e-05, 1.2516975e-06],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00]], dtype

Epoch 1/2
938/938 [==============================] - 4s 4ms/step - loss: 1.4986 - accuracy: 0.9642 - val_loss: 1.4951 - val_accuracy: 0.9671
Epoch 2/2
938/938 [==============================] - 4s 4ms/step - loss: 1.4911 - accuracy: 0.9716 - val_loss: 1.4972 - val_accuracy: 0.9653
final pruning and eval
Epoch 1/100
938/938 [==============================] - 3s 4ms/step - loss: 1.4835 - accuracy: 0.9789 - val_loss: 1.4886 - val_accuracy: 0.9733
Epoch 2/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4793 - accuracy: 0.9828 - val_loss: 1.4871 - val_accuracy: 0.9749
Epoch 3/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4777 - accuracy: 0.9842 - val_loss: 1.4868 - val_accuracy: 0.9753
Epoch 4/100
938/938 [==============================] - 4s 4ms/step - loss: 1.4755 - accuracy: 0.9863 - val_loss: 1.4835 - val_accuracy: 0.9786
Epoch 5/100
938/938 [==============================] - 3s 4ms/step - loss: 1.4746 - accuracy: 0.9871 - val_loss: 1.4854 - v

...


5it [1:09:31, 836.79s/it]

self.p 2
flatten(p) TensorFlowTensor(<tf.Tensor: shape=(500, 784), dtype=float32, numpy=
                 array([[4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        ...,
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07]], dtype

Epoch 1/2
938/938 [==============================] - 3s 3ms/step - loss: 1.4984 - accuracy: 0.9645 - val_loss: 1.4961 - val_accuracy: 0.9660
Epoch 2/2
938/938 [==============================] - 3s 3ms/step - loss: 1.4904 - accuracy: 0.9720 - val_loss: 1.4915 - val_accuracy: 0.9715
Epoch 1/2
938/938 [==============================] - 3s 3ms/step - loss: 1.4832 - accuracy: 0.9792 - val_loss: 1.4870 - val_accuracy: 0.9751
Epoch 2/2
938/938 [==============================] - 3s 3ms/step - loss: 1.4794 - accuracy: 0.9829 - val_loss: 1.4855 - val_accuracy: 0.9763
Epoch 1/2
938/938 [==============================] - 3s 4ms/step - loss: 1.4794 - accuracy: 0.9849 - val_loss: 1.4841 - val_accuracy: 0.9785
Epoch 2/2
938/938 [==============================] - 3s 4ms/step - loss: 1.4746 - accuracy: 0.9882 - val_loss: 1.4839 - val_accuracy: 0.9776
Epoch 1/2
938/938 [==============================] - 3s 4ms/step - loss: 1.4827 - accuracy: 0.9844 - val_loss: 1.4838 - val_accuracy: 0.9799
Epoch 2/2
938


6it [1:28:23, 883.92s/it]
100%|██████████| 1/1 [1:28:23<00:00, 5303.50s/it]

self.p 2
flatten(p) TensorFlowTensor(<tf.Tensor: shape=(500, 784), dtype=float32, numpy=
                 array([[4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        ...,
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [4.7683716e-07, 4.7683716e-07, 4.7683716e-07, ..., 4.7683716e-07,
                         4.7683716e-07, 4.7683716e-07],
                        [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
                         0.0000000e+00, 0.0000000e+00]], dtype

In [46]:
import json
with open('test.json', 'w') as f:
    json.dump(new_list, f)
with open('test.json', 'r') as f:
    test = json.load(f)
print(test)

[[0.398, 0.5, 0.534, 0.534, 0.626, 0.638], [0.398, 0.5, 0.534, 0.534, 0.626, 0.638]]


In [2]:
def get_average_success_rates(all_sucess_rates):
    success_per_pruning_rate=[]
    for i in range(len(all_success_rates)):
        for j in range(len(all_success_rates[i])):

            try:
                success_per_pruning_rate[j].append(all_success_rates[i][j])
            except:
                success_per_pruning_rate.append([])
                success_per_pruning_rate[j].append(all_success_rates[i][j])
    avg_success_per_pruning_rate = [sum(x)/len(x) for x in success_per_pruning_rate]
    return avg_success_per_pruning_rate

In [3]:
def get_average_accuracies(all_accuracies):
    acc_per_pruning_rate=[]
    for i in range(len(all_accuracies)):
        for j in range(len(all_accuracies[i])):

            try:
                acc_per_pruning_rate[j].append(all_accuracies[i][j][1])
            except:
                acc_per_pruning_rate.append([])
                acc_per_pruning_rate[j].append(all_accuracies[i][j][1])
    avg_acc_per_pruning_rate = [sum(x)/len(x) for x in acc_per_pruning_rate]
    return avg_acc_per_pruning_rate

# Helper Functions

In [4]:
def train_model(model, to_convergence=True):
    if to_convergence == True:
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=100,
            callbacks=[callback],
            validation_data=(x_test, y_test),
            )
    if to_convergence == False:
        model.fit(
            x=x_train,
            y=y_train,
            batch_size=64,
            epochs=2,
            validation_data=(x_test, y_test),
            )
    return model

def prune_weights(model, pruning_ratio):
    weights = model.get_weights()
    weights_to_prune = model.get_weights()
    for index, weight in enumerate(weights):
        if (index == 0) or (index == 3) or (index == 6):
            flat_weights = weight.flatten()
            flat_weights_to_prune = weights_to_prune[index+1].flatten()
            #print (flat_weights_to_prune.shape, flat_weights.shape)
            flat_weights_df = pd.DataFrame(flat_weights)
            flat_weights_to_prune_df = pd.DataFrame(flat_weights_to_prune)
            no_of_weights_to_prune = int(len(flat_weights)*pruning_ratio)
            #print(no_of_weights_to_prune)
            indices_to_delete = flat_weights_df.abs().values.argsort(0)[:no_of_weights_to_prune]
            for idx_to_delete in indices_to_delete:
                flat_weights_to_prune[idx_to_delete] = 0
            dims = weights_to_prune[index+1].shape
            weights_reshaped = flat_weights_to_prune.reshape(dims)
            weights_to_prune[index+1] = weights_reshaped
    #print(weights_to_prune)
    return weights_to_prune

def pgd_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.LinfProjectedGradientDescentAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[15/255]
    )
    return np.count_nonzero(adversarials[2])/len(y)

def cw2_attack(model_to_attack):
    fmodel = fb.models.TensorFlowModel(model_to_attack, bounds=(0,1))
    attack = fb.attacks.L2CarliniWagnerAttack()
    adversarials = attack(
        fmodel,
        x,
        y,
        epsilons=[.5]
    )
    return np.count_nonzero(adversarials[2])/len(y)

# Load Data

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

x = tf.convert_to_tensor(x_train[:500].reshape(500,28*28))
y = tf.convert_to_tensor([y_train[:500]])[0];

# Define Model

In [74]:
class CustomLayer(layers.Layer):

    def __init__(self, units=32, activation='relu'):
        super(CustomLayer, self).__init__()
        self.units = units
        self.activation = activation

    def build(self, input_shape):
        #print(input_shape)
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True,
                                name='unpruned_weights')
        self.mask = self.add_weight(shape=(self.w.shape),
                                    initializer='ones',
                                    trainable=False,
                                   name='pruning_mask')
        self.pruned_w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='ones',
                                 trainable=False,
                                       name='pruned_weights')
        
    def call(self, inputs):
        #self.mask_2 = tf.multiply(self.mask, self.mask_2)
        self.pruned_w = tf.multiply(self.w, self.mask)
        #print(self.pruned_w.eval())
        x = tf.matmul(inputs, self.pruned_w)
        
        if self.activation == 'relu':
            return tf.keras.activations.relu(x)
        if self.activation == 'softmax':
            return tf.keras.activations.softmax(x)
        raise ValueError('Activation function not implemented')

class LeNet300_100(tf.keras.Model):
    def __init__(self):
        super(LeNet300_100, self).__init__()
        self.dense1 = CustomLayer(300)
        self.dense2 = CustomLayer(100)
        self.dense3 = CustomLayer(10, activation='softmax')
        
    def call(self, inputs):
        
        x = self.dense1(inputs)
        #print(x.shape)
        x = self.dense2(x)
        #print(x.shape)
        return self.dense3(x)
    

# Compile and Train Model

In [75]:
model = LeNet300_100()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) ,
              metrics=['accuracy'],
              experimental_run_tf_function=False
              
             )

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(x=x_train,
          y=y_train,
          batch_size=64,
          epochs=1,
          callbacks=[callback],
          validation_data=(x_test, y_test),
         )
model.save('./saved-models/mini-pipeline-mlp-baseline-model')

938/938 [==============================] - 5s 5ms/step - loss: 1.5740 - accuracy: 0.9015 - val_loss: 1.5219 - val_accuracy: 0.9433
INFO:tensorflow:Assets written to: ./saved-models/mini-pipeline-mlp-baseline-model/assets
